*Read data form bronze layer*

In [0]:
%sql
SELECT * FROM bara_slaes_project.bronze.sales_details  LIMIT 5;

In [0]:
df=spark.table("bara_slaes_project.bronze.sales_details")

---
*Data Transformation*

In [0]:
# *Rename Cols*
rename_mapping = {
    'sls_ord_num':'Order_Number',
    'sls_prd_key':'Product_Key',
    'sls_cust_id':'Customer_ID',
    'sls_order_dt':'Order_Date',
    'sls_ship_dt':'Ship_Date',
    'sls_due_dt':'Due_Date',
    'sls_sales':'Sales',
    'sls_quantity':'Quantity',
    'sls_price':'Price'
}

df = df.withColumnsRenamed(rename_mapping)

In [0]:

## Trim
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import *
for field in df.schema.fields:
    if isinstance(field.dataType,StringType):
        df=df.withColumn(field.name,trim(col(field.name)))

In [0]:
df= (
    df
    .withColumn("Price",
              when(col("Price").isNull(),
                   when(
                       col("Quantity")!=0,
                       col("Sales")/col("Quantity")
                   ).otherwise("None")
              ).otherwise(col("Price"))
    )
    .withColumn("Sales",
              when(col("Sales").isNull(),
                   when(
                       col("Quantity")!=0,
                       col("Price")*col("Quantity")
                   ).otherwise("None")
              ).otherwise(col("Sales"))
    )     
)                 

In [0]:
df = (
    df
    .withColumn(
        "Order_Date",
        F.when(
            (col("Order_Date") == 0) | (length(col("Order_Date")) != 8),
            None
        ).otherwise(F.to_date(col("Order_Date").cast("string"), "yyyyMMdd"))
    )
    .withColumn(
        "Ship_Date",
        F.when(
            (col("Ship_Date") == 0) | (length(col("Ship_Date")) != 8),
            None
        ).otherwise(F.to_date(col("Ship_Date").cast("string"), "yyyyMMdd"))
    )
    .withColumn(
        "Due_Date",
        F.when(
            (col("Due_Date") == 0) | (length(col("Due_Date")) != 8),
            None
        ).otherwise(F.to_date(col("Due_Date").cast("string"), "yyyyMMdd"))
    )
)
    

---
*Write into Silver layer*

In [0]:
df.write.mode("overwrite")\
    .format("delta")\
        .saveAsTable("bara_slaes_project.silver.Sales_info")

In [0]:
%sql
SELECT * FROM bara_slaes_project.silver.Sales_info  LIMIT 5;